In [1]:
import en_core_web_md
import modin.pandas as pd
import os
import re
import spacy

nlp = en_core_web_md.load()
data_dir = '/Users/dashiell/reddit_blm/'
def load_data():
    return pd.concat(
        [pd.read_csv(data_dir + fp) for fp in os.listdir(data_dir)]
    )
df = load_data()

def break_out_links(text):
    link_pattern = '\[(.*?)\]\((.*?)\)'
    matches = [m for m in re.finditer(link_pattern, text)]
    offset = 0
    for m in matches:
        text = text[:m.start(0)-offset] + ' ' + m.group(1) + ' ' + m.group(2) + ' ' + text[m.end(0)-offset:]
        offset += 1
    return text

def unescape_chars(text):
    ntext = re.sub(r'\\\*', '', text)
    ntext = re.sub(r'&gt;', '>', ntext)
    ntext = re.sub(r'&lt;', '<', ntext)
    ntext = re.sub(r'&amp;', '&', ntext)
    ntext = re.sub(r'\\~', '~', ntext)
    return re.sub(r"\\'", "'", ntext)


In [17]:
comments = df.body.str.replace('&com;', ',').str.replace('&nl;', '\n').values

In [18]:
test_doc = nlp(comments[0])


In [23]:
for ent in test_doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [24]:
test_doc

It has nothing to do with drownings voices, it happend because people complained about the sentence is racist and created an alternative and it is a racist sentence, they attack white/latinand use racial slurs against anyone not black. So yes they are extremely racist. And let's for a moment say All Lives Matter is realy a conspiracy to drown voices so why not black lives matter too?

In [92]:
reddit_quote_pattern = re.compile('&gt;(.*)\\n\\n')
test = re.match(reddit_quote_pattern, comments[212])
comments[1]

'&gt;[Obviously "all"\\* lives matter. No one said they didn\'t.](https://i.imgur.com/UyCmhSx.png)\n\n[Bernie Sanders said they didn\'t](https://youtu.be/XzNmcyic74A) when he was asked point blank.\n\n&gt;However, data shows that relative to the percentage of the population they represent, \\*\\*the rate of black American deaths from police shootings is \\~2.5-3x that of white Americans deaths\\*\\*\n\nSo what? You act like we have to pick and choose whose lives to save. We can solve this problem for EVERYONE instead of only solving it for one race.'

In [110]:
m = re.findall('&gt;.*?\\n\\n', comments[1])

In [125]:
x = comments[1]

In [137]:
link_pattern = '\[(.*?)\]\((.*?)\)'
for y in re.finditer(link_pattern, x):
    print(y.start(0))
    print(y.end(0))
    print(y.group(0))
    print(y.group(1))

4
95
[Obviously "all"\* lives matter. No one said they didn't.](https://i.imgur.com/UyCmhSx.png)
Obviously "all"\* lives matter. No one said they didn't.
97
160
[Bernie Sanders said they didn't](https://youtu.be/XzNmcyic74A)
Bernie Sanders said they didn't


In [139]:
matches = [y for y in re.finditer(link_pattern, x)]

In [141]:
first = matches[0]

In [143]:
first.group(1)

'Obviously "all"\\* lives matter. No one said they didn\'t.'

In [145]:
x[:first.start(0)] + ' ' + first.group(1) + ' ' + first.group(2) + ' ' + x[first.end(0):]

'&gt; Obviously "all"\\* lives matter. No one said they didn\'t. https://i.imgur.com/UyCmhSx.png \n\n[Bernie Sanders said they didn\'t](https://youtu.be/XzNmcyic74A) when he was asked point blank.\n\n&gt;However, data shows that relative to the percentage of the population they represent, \\*\\*the rate of black American deaths from police shootings is \\~2.5-3x that of white Americans deaths\\*\\*\n\nSo what? You act like we have to pick and choose whose lives to save. We can solve this problem for EVERYONE instead of only solving it for one race.'

In [153]:
comments[3456]

'All lives matter, he writes on a sign that he just made using materials he bought from target just hours before while not wearing a mask to protect his grandma'

In [234]:
def break_out_links(text):
    link_pattern = '\[(.*?)\]\((.*?)\)'
    offset = 0
    for m in re.finditer(link_pattern, text):
        text = text[:m.start(0)-offset] + ' ' + m.group(1) + ' ' + m.group(2) + ' ' + text[m.end(0)-offset:]
        offset += 1
    return text


def break_out_quotes(text):
    quote_pattern = '>(.*?)\\n\\n'
    quote_replacement = ' REDDITQUOTE '
    offset = 0
    quotes = []
    for m in re.finditer(quote_pattern, text):
        print(offset)
        text = text[:m.start(0)-offset] + quote_replacement + text[m.end(0)-offset:]
        quotes.append(m.group(1))
        offset += len(m[0]) - len(quote_replacement)
    return text, quotes

    
def unescape_chars(text):
    ntext = re.sub(r'\\\*', '', text)
    ntext = re.sub(r'&gt;', '>', ntext)
    ntext = re.sub(r'&lt;', '<', ntext)
    ntext = re.sub(r'&amp;', '&', ntext)
    ntext = re.sub(r'\\~', '~', ntext)
    return re.sub(r"\\'", "'", ntext)

In [235]:
x = comments[1]

In [236]:
break_out_quotes(break_out_links(unescape_chars(x)))

0
78


(" REDDITQUOTE  Bernie Sanders said they didn't https://youtu.be/XzNmcyic74A  when he was asked point blank.\n\n REDDITQUOTE So what? You act like we have to pick and choose whose lives to save. We can solve this problem for EVERYONE instead of only solving it for one race.",
 [' Obviously "all" lives matter. No one said they didn\'t. https://i.imgur.com/UyCmhSx.png ',
  'However, data shows that relative to the percentage of the population they represent, the rate of black American deaths from police shootings is ~2.5-3x that of white Americans deaths'])

In [173]:
break_out_links(x)

'&gt; Obviously "all"\\* lives matter. No one said they didn\'t. https://i.imgur.com/UyCmhSx.png \n\n Bernie Sanders said they didn\'t https://youtu.be/XzNmcyic74A  when he was asked point blank.\n\n&gt;However, data shows that relative to the percentage of the population they represent, \\*\\*the rate of black American deaths from police shootings is \\~2.5-3x that of white Americans deaths\\*\\*\n\nSo what? You act like we have to pick and choose whose lives to save. We can solve this problem for EVERYONE instead of only solving it for one race.'